In [1]:
import numpy as np
from sklearn import metrics
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from sklearn.datasets import make_blobs
from sklearn.preprocessing import StandardScaler
import pandas as pd
from astropy.table import Table, vstack
import hdbscan
from pathlib import Path
import seaborn as sns
import glob
import json
%matplotlib inline
sns.set_color_codes()

In [2]:
ROOT_PATH = Path("..") 

In [3]:
table1 = Table.read("Halpha-DR3_noFlag_3ferr_16r.ecsv", format="ascii.ecsv")
table2 = Table.read("Halpha-DR3_noFlag_3ferr_16r18.ecsv", format="ascii.ecsv")
table3 = Table.read("Halpha-DR3_noFlag_3ferr_18r20.ecsv", format="ascii.ecsv")
table4 = Table.read("Halpha-DR3_noFlag_3ferr_20r21.ecsv", format="ascii.ecsv")

In [4]:
# Merge tall four tables
table_merge = vstack([table1, table2, table3, table4])
table_merge.colnames

['Unnamed: 0',
 'Field',
 'ID',
 'RA',
 'DEC',
 'FWHM',
 'FWHM_n',
 'A',
 'B',
 'ISOarea',
 'KRON_RADIUS',
 'PhotoFlagDet',
 's2n_Det_iso',
 'U_PStotal',
 'F378_PStotal',
 'F395_PStotal',
 'F410_PStotal',
 'F430_PStotal',
 'G_PStotal',
 'F515_PStotal',
 'R_PStotal',
 'F660_PStotal',
 'I_PStotal',
 'F861_PStotal',
 'Z_PStotal',
 'e_U_PStotal',
 'e_F378_PStotal',
 'e_F395_PStotal',
 'e_F410_PStotal',
 'e_F430_PStotal',
 'e_G_PStotal',
 'e_F515_PStotal',
 'e_R_PStotal',
 'e_F660_PStotal',
 'e_I_PStotal',
 'e_F861_PStotal',
 'e_Z_PStotal',
 'F378_iso',
 'F395_iso',
 'F410_iso',
 'F430_iso',
 'G_iso',
 'F515_iso',
 'R_iso',
 'F660_iso',
 'I_iso',
 'F861_iso',
 'Z_iso',
 'e_U_iso',
 'e_F378_iso',
 'e_F395_iso',
 'e_F410_iso',
 'e_F430_iso',
 'e_G_iso',
 'e_F515_iso',
 'e_R_iso',
 'e_F660_iso',
 'e_I_iso',
 'e_F861_iso',
 'e_Z_iso',
 'r - i',
 'r - J0660',
 'e(r - i)',
 'e(r - J0660)',
 'col1']

In [5]:
table_merge

Unnamed: 0,Field,ID,RA,DEC,FWHM,FWHM_n,A,B,ISOarea,KRON_RADIUS,PhotoFlagDet,s2n_Det_iso,U_PStotal,F378_PStotal,F395_PStotal,F410_PStotal,F430_PStotal,G_PStotal,F515_PStotal,R_PStotal,F660_PStotal,I_PStotal,F861_PStotal,Z_PStotal,e_U_PStotal,e_F378_PStotal,e_F395_PStotal,e_F410_PStotal,e_F430_PStotal,e_G_PStotal,e_F515_PStotal,e_R_PStotal,e_F660_PStotal,e_I_PStotal,e_F861_PStotal,e_Z_PStotal,F378_iso,F395_iso,F410_iso,F430_iso,G_iso,F515_iso,R_iso,F660_iso,I_iso,F861_iso,Z_iso,e_U_iso,e_F378_iso,e_F395_iso,e_F410_iso,e_F430_iso,e_G_iso,e_F515_iso,e_R_iso,e_F660_iso,e_I_iso,e_F861_iso,e_Z_iso,r - i,r - J0660,e(r - i),e(r - J0660),col1
int64,str13,str25,float64,float64,float64,float64,float64,float64,int64,float64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64
488,HYDRA-0011,iDR3.HYDRA-0011.025139,151.42202375431012,-24.1683267505366,1.8182144,0.89325166,1.2579726,0.9334866,20,0.0,3,479.39966,17.217575,16.826605,16.567743,16.096035,15.93552,16.698824,15.602974,15.852499,15.269681,15.20847,15.132599,15.11796,0.010222789,0.0122633325,0.015012143,0.009821964,0.008902777,0.00552345,0.0062212637,0.0032707807,0.0037926577,0.0025150075,0.005356403,0.0030152262,17.252916,16.945045,16.463615,16.33493,16.895357,15.944144,16.079996,15.546656,15.498211,15.424429,15.42496,0.010342245,0.012281164,0.015075396,0.009964665,0.009128071,0.005182142,0.006317072,0.0032160366,0.0038400677,0.002561187,0.005432999,0.0030626028,0.6440289999999997,0.5828179999999996,0.004125926455058153,0.005008219126282493,--
1355,HYDRA-0011,iDR3.HYDRA-0011.066566,150.3235245777253,-23.175435217832614,4.1982293,2.0625045,2.7849326,2.2189088,298,2.5925863,2,655.8235,16.973087,19.854927,16.42228,16.014027,15.531513,18.108053,15.184443,14.406248,14.369801,17.369287,17.448772,14.146488,0.008882983,0.10042154,0.013738524,0.009568994,0.0073562614,0.012113465,0.0051423754,0.0016706635,0.002513905,0.008072701,0.02204594,0.0018747156,23.464338,17.413021,16.495142,15.81933,17.44698,15.451651,14.53604,14.490839,16.779366,16.98602,14.180124,0.020869005,7.9428244,0.054643247,0.021541478,0.012214524,0.013826787,0.007450805,0.0017778614,0.0027357808,0.010475083,0.04035355,0.0023250296,-2.963039,0.03644699999999901,0.008243762367125417,0.003018415955307891,--
1508,HYDRA-0011,iDR3.HYDRA-0011.073716,151.02535350205395,-23.166713931002704,4.4331264,2.1779046,1.8904788,1.6889452,117,2.7351406,2,353.2615,17.381468,19.812702,16.889364,16.536507,16.206982,20.41994,16.76994,15.57645,15.293367,18.143543,18.098372,15.369935,0.011128998,0.0968434,0.017961053,0.012661303,0.0104237115,0.062161766,0.011649385,0.0029048836,0.0038925712,0.013179879,0.03571828,0.0034732756,99.0,17.729055,17.087479,16.587132,19.175808,17.033707,15.725418,15.442906,17.831266,18.129858,15.528647,0.019386353,21.507524,0.048044194,0.024401002,0.016155828,0.040358566,0.018215498,0.003139965,0.00421774,0.017294208,0.07260927,0.0047120354,-2.5670930000000016,0.28308299999999953,0.013496205362404277,0.004857001057918188,--
10456,HYDRA-0017,iDR3.HYDRA-0017.000100,159.7456097449217,-24.61822082802332,6.4657593,1.6452757,3.6041832,2.669173,399,2.8504643,0,851.72485,19.532448,18.205341,19.028248,17.966213,17.56451,15.107697,16.82592,14.00636,13.593677,14.389355,13.129466,13.249959,0.05769594,0.033099893,0.09335109,0.040311623,0.02903483,0.003157956,0.01545036,0.0017410432,0.0017861181,0.00223182,0.002152438,0.0015608366,19.620228,99.0,18.095419,17.539497,15.312523,16.736454,14.22501,13.630016,14.436251,13.100569,13.417247,2.2116404,0.25525692,20.667572,0.09736602,0.05764188,0.0036726033,0.024189238,0.0017717676,0.0014690759,0.0028122994,0.0021013308,0.0021267

In [6]:
table_merge.remove_columns(['Unnamed: 0', 'col1'])

In [7]:
table_merge.write("Halpha-DR3_noFlag_3ferr_merge.ecsv", format="ascii.ecsv")
table_merge.write("Halpha-DR3_noFlag_3ferr_merge.dat", format="ascii.commented_header")